In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml

### Get player position info from site below

In [7]:
test = requests.get('https://www.pro-football-reference.com/years/2010/fantasy.htm')

In [8]:
test

<Response [200]>

In [10]:
soup = BeautifulSoup(test.text, 'lxml')

In [13]:
table = soup.find_all('table')[0]

In [19]:
trs = table.find_all('tr')
len(trs)

599

In [31]:
players = {}

In [32]:
for tr in trs[2:]:
    row = []
    counter = 0
    for td in tr.find_all('td'):
        if counter == 0:
            player = td.text.replace('*', '').replace('+', '')
        elif counter == 2:
            pos = td.text.replace('*', '').replace('+', '')
            print('adding', player, ': ', pos)
            players[player] = [pos]
        else:
            pass
        counter += 1
        
        

adding Arian Foster :  RB
adding Peyton Hillis :  RB
adding Adrian Peterson :  RB
adding Jamaal Charles :  RB
adding Chris Johnson :  RB
adding Brandon Lloyd :  WR
adding Dwayne Bowe :  WR
adding Darren McFadden :  RB
adding Roddy White :  WR
adding Michael Vick :  QB
adding Rashard Mendenhall :  RB
adding LeSean McCoy :  RB
adding Greg Jennings :  WR
adding Matt Forte :  RB
adding Michael Turner :  RB
adding Ray Rice :  RB
adding Aaron Rodgers :  QB
adding Mike Wallace :  WR
adding Calvin Johnson :  WR
adding Tom Brady :  QB
adding Ahmad Bradshaw :  RB
adding Maurice Jones-Drew :  RB
adding Steven Jackson :  RB
adding Jason Witten :  TE
adding Philip Rivers :  QB
adding Peyton Manning :  QB
adding Reggie Wayne :  WR
adding Hakeem Nicks :  WR
adding Andre Johnson :  WR
adding Steve Johnson :  WR
adding BenJarvus Green-Ellis :  RB
adding Mike Williams :  WR
adding Miles Austin :  WR
adding Jeremy Maclin :  WR
adding Drew Brees :  QB
adding Antonio Gates :  TE
adding Cedric Benson :  RB


adding Brian Brohm :  QB
adding Todd Collins :  QB


In [50]:
df = pd.DataFrame.from_records(players).T
df.head()

,0
Aaron Brown,RB
Aaron Hernandez,TE
Aaron Rodgers,QB
Adrian Arrington,WR
Adrian Peterson,RB


In [51]:
df = df.reset_index()
df.head()

,index,0
0,Aaron Brown,RB
1,Aaron Hernandez,TE
2,Aaron Rodgers,QB
3,Adrian Arrington,WR
4,Adrian Peterson,RB


In [52]:
df.columns = ['Player', 'Position']

In [53]:
df.head()

,Player,Position
0,Aaron Brown,RB
1,Aaron Hernandez,TE
2,Aaron Rodgers,QB
3,Adrian Arrington,WR
4,Adrian Peterson,RB


### now we can do this for all 8 years

In [54]:
base_url = 'https://www.pro-football-reference.com/years/'
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
suffix = '/fantasy.htm'

In [55]:
all_players = {}

In [60]:
for year in years:
    
    url = base_url + year + suffix
    print(url)
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        table = soup.find_all('table')[0]
        trs = table.find_all('tr')
        
        for tr in trs[2:]:
            counter = 0
            for td in tr.find_all('td'):
                if counter == 0:
                    player = td.text.replace('*', '').replace('+', '')
                elif counter == 2:
                    pos = td.text.replace('*', '').replace('+', '')
                    all_players[player] = [pos]
                else:
                    pass
                counter += 1
        
    else:
        print(r.status_code)
        raise BaseException
        

https://www.pro-football-reference.com/years/2010/fantasy.htm
https://www.pro-football-reference.com/years/2011/fantasy.htm
https://www.pro-football-reference.com/years/2012/fantasy.htm
https://www.pro-football-reference.com/years/2013/fantasy.htm
https://www.pro-football-reference.com/years/2014/fantasy.htm
https://www.pro-football-reference.com/years/2015/fantasy.htm
https://www.pro-football-reference.com/years/2016/fantasy.htm
https://www.pro-football-reference.com/years/2017/fantasy.htm


In [63]:
df = pd.DataFrame.from_records(all_players).T
df = df.reset_index()
df.columns = ['Player', 'Position']
df.head()

,Player,Position
0,A.J. Derby,TE
1,A.J. Feeley,QB
2,A.J. Green,WR
3,A.J. Jenkins,WR
4,A.J. McCarron,QB


In [64]:
df.shape

(1592, 2)

In [67]:
df.to_csv('./data/player_pos.csv', index=None)

### There are still 320 players from this time period with no position info, but it is safe to assume they had little to no impact in the fantasy realm and can excluded from the data set